In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 데이터 생성 및 초기 설정

In [ ]:
data_num=10
data_len=100
distance=10
pattern_data=[]
polarization_pattern_data=[]
for i in range(data_num):
    data=np.random.randint(0,2,size=data_len)
    pattern_data.append(np.array(data).reshape(1,data_len))
    polarization_pattern_data.append(np.array(2*data-1).reshape(1,data_len))


# 홉필드 네트워크 구현

In [ ]:
class HopfieldNetwork:
    def __init__(self):
        self.t = np.array([0]*data_len*data_len).reshape(data_len,data_len)

    def learn_t(self, polarization_pattern_data):
        for i in range(data_num):
            self.t=self.t+np.dot(polarization_pattern_data[i].transpose(),polarization_pattern_data[i])
        self.t=self.t-np.identity(data_len)*data_num
        

    def noise_generator(self,distance, pattern_data):
        noise_v=pattern_data[0].copy()
        for i in range(distance):
            noise_index=np.random.randint(0,data_len)
            noise_v[0][noise_index]=(noise_v[0][noise_index]-1)*(-1)
        return noise_v   
                           
    def evaluate(self,noise_v,pattern_data):
        count=0
        energy_list=[]
        for i in range(data_len):
            if noise_v[0][i]==pattern_data[0][0][i]:
                count+=1
        for i in range(data_num):
            energy=self.energy_evaluation(pattern_data[i])
            energy_list.append(energy)
        noise_energy=self.energy_evaluation(noise_v)
        distinction=noise_energy-energy_list[0]
       
        return count, distinction
    
    def energy_evaluation(self,data):
        energy=(-0.5)*np.dot(np.dot(data,self.t),data.transpose())[0][0]
        
        return energy
                           
    def lenrn_hopfield(self ,pattern_data,polarization_pattern_data, time):
        
        self.learn_t(polarization_pattern_data)
        
        noise_v=self.noise_generator(distance,pattern_data)
        time_list=[]
        count_list=[]
        distinction_list=[]
        for i in range(time):
            time_list.append(i)
            random_index=np.random.randint(0,data_len)
            tv=np.dot(self.t,noise_v.transpose())
            if tv[random_index][0]>=0:
                noise_v[0][random_index]=1
            elif tv[random_index][0]< 0:
                noise_v[0][random_index]=0
            count,distinction=self.evaluate(noise_v,pattern_data)
            count_list.append(data_len-count)
            distinction_list.append(distinction)
            if count==data_len:
                break
    

        plt.plot(time_list,count_list)
        plt.xlabel("time")
        plt.ylabel("distance")
        plt.show()
        plt.plot(time_list,distinction_list)
        plt.xlabel("time")
        plt.ylabel("energy distinction")
        plt.show()
            

# 코드 실행

In [ ]:
run=HopfieldNetwork()

In [ ]:
run.lenrn_hopfield(pattern_data,polarization_pattern_data,1000)